In [4]:
import sys
import csv

maxInt = sys.maxsize
csv.field_size_limit(maxInt)
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd

with open(
    "./verbalist/datasets/ficbook/fic_anx_gen 3.csv", "r", encoding="utf-8"
) as file:
    reader = csv.DictReader(file)
    df = pd.DataFrame(reader)
    df["likes"] = df["likes"].astype(int)
# dataset = load_dataset("dim/ficbook_raw")
# dataset = dataset['train']
# dataset

In [5]:
dataset = Dataset.from_pandas(df)
dataset = dataset.sort(column_names="likes", reverse=True)
dataset = dataset.filter(
    lambda x: len(x["text"]) > 0
    and len(x["title"]) > 0
    and len(x["description"]) > 0
    and len(x["tag"]) > 0
)

Filter:   0%|          | 0/114411 [00:00<?, ? examples/s]

In [6]:
dataset[0]

{'id': '1',
 'author': 'BajHu',
 'title': 'Метро',
 'link': '',
 'description': '"Метро в час-пик - самый слэшерский вид транспорта."(с) Нэлт "ЛиЛ в современном мире". Или фемслэшерский?)',
 'tag': 'PWP',
 'likes': 25398,
 'date': '2011-11-24',
 'review': '0',
 'format': '',
 'text': 'Стук колес поезда отдавался в ушах гулким шумом, перекрывая собой звучащую фоном музыку. Марина зажмурилась на мгновенье, а потом снова попыталась сосредоточиться на предложении, которое читала уже в пятый раз. Буквы не желали складываться в слова. Марина вздохнула и тут же чуть не подавилась воздухом, потому что поезд вдруг начал резко тормозить. На благо, упасть в вагоне было некуда, все-таки вечерний час-пик. Девушка выдохнула, стараясь одной рукой одернуть чуть задравшуюся юбку.\n«На кой черт я ее надела?!» - Марина бросила бесплодные попытки и снова уткнулась в книжку.\nСначала она подумала, что ей показалось, что кто-то коснулся ее бедра. Ну, мало ли чего в толпе ни случается. Но уже через пару секу

In [7]:
dataset = dataset.select(range(10000))

In [8]:
dataset["likes"][:100]

[25398,
 16926,
 15015,
 13531,
 11707,
 10725,
 9120,
 8937,
 8520,
 8410,
 8167,
 8156,
 8014,
 7811,
 7610,
 7543,
 7308,
 7028,
 7013,
 6835,
 6463,
 6374,
 6371,
 6353,
 6272,
 6228,
 6217,
 6141,
 6078,
 6039,
 6024,
 5936,
 5933,
 5895,
 5812,
 5790,
 5727,
 5498,
 5472,
 5461,
 5451,
 5216,
 5091,
 4965,
 4955,
 4928,
 4925,
 4924,
 4859,
 4804,
 4783,
 4725,
 4714,
 4642,
 4638,
 4621,
 4490,
 4425,
 4373,
 4230,
 4210,
 4205,
 4162,
 4120,
 4093,
 4056,
 4028,
 4024,
 4019,
 3985,
 3984,
 3979,
 3967,
 3951,
 3879,
 3840,
 3832,
 3827,
 3814,
 3783,
 3746,
 3727,
 3681,
 3610,
 3578,
 3559,
 3529,
 3483,
 3476,
 3408,
 3381,
 3373,
 3327,
 3314,
 3306,
 3299,
 3284,
 3276,
 3256,
 3254]

In [52]:
# dataset.push_to_hub("dim/ficbook_raw_best_10k", private=True)

### prompts


In [54]:
dataset = load_dataset("dim/ficbook_raw_best_10k")
dataset = dataset["train"]
dataset

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--ficbook_raw_best_10k-6b4f744f1834865d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'author', 'title', 'link', 'description', 'tag', 'likes', 'date', 'review', 'format', 'text', 'rating', 'status', 'parts'],
    num_rows: 10000
})

In [58]:
dataset[0]

{'id': '1',
 'author': 'BajHu',
 'title': 'Метро',
 'link': '',
 'description': '"Метро в час-пик - самый слэшерский вид транспорта."(с) Нэлт "ЛиЛ в современном мире". Или фемслэшерский?)',
 'tag': 'PWP',
 'likes': 25398,
 'date': '2011-11-24',
 'review': '0',
 'format': '',
 'text': 'Стук колес поезда отдавался в ушах гулким шумом, перекрывая собой звучащую фоном музыку. Марина зажмурилась на мгновенье, а потом снова попыталась сосредоточиться на предложении, которое читала уже в пятый раз. Буквы не желали складываться в слова. Марина вздохнула и тут же чуть не подавилась воздухом, потому что поезд вдруг начал резко тормозить. На благо, упасть в вагоне было некуда, все-таки вечерний час-пик. Девушка выдохнула, стараясь одной рукой одернуть чуть задравшуюся юбку.\n«На кой черт я ее надела?!» - Марина бросила бесплодные попытки и снова уткнулась в книжку.\nСначала она подумала, что ей показалось, что кто-то коснулся ее бедра. Ну, мало ли чего в толпе ни случается. Но уже через пару секу

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")

In [10]:
len(tokenizer(dataset[0]["text"])["input_ids"])

1727

In [11]:
from tqdm import tqdm

prompts = []

for item in tqdm(dataset):
    description = item["description"]
    title = item["title"]
    tag = item["tag"]
    text = item["text"]
    prompt = f"напиши фанфик с названием {title} и следующим описанием {description}, с тегами {tag}"

    prompt_len = len(tokenizer(prompt, add_special_tokens=False)["input_ids"])
    solution_len = 2048 - 100
    solution_len -= prompt_len
    text_input_ids = tokenizer(text, add_special_tokens=False)["input_ids"]
    text_input_ids = text_input_ids[:solution_len]
    decoded_text_len = len(tokenizer.decode(text_input_ids))

    solution = text[:decoded_text_len]
    prompts.append(
        {
            "prompt": prompt,
            "solution_short_llama2": solution,
            "solution_full": text,
        }
    )
    # break

100%|██████████| 10000/10000 [04:09<00:00, 40.04it/s]


In [12]:
prompts = Dataset.from_list(prompts)

In [13]:
prompts[30]

{'prompt': 'напиши фанфик с названием Dirty-talk и следующим описанием На заявку кинк-феста 14.81 "Грязные разговоры", шепот на ухо. , с тегами PWP,Нецензурная лексика,ООС,Юмор',
 'solution_short_llama2': '- «Что на тебе сейчас надето, детка? О, черные кружевные трусики.  Обожаю, когда ты в них. Ты медленно снимаешь...» тьфу, чертовщина, - Холмс брезгливо отбросил от себя несколько блокнотных листков. - Я начинаю понимать убийцу. Другого ничего нет?..\nСидящий напротив Джон оторвался от экрана ноута и саркастически ухмыльнулся.\n- Извини, нет. Моя, и без того идиотская, фантазия на этом иссякла. Почему бы тебе не спросить у настоящих работников этой службы, как они... ну...\n- А разве ты там не работаешь? - светским тоном подколол его Холмс и едва увернулся от полетевшей в него диванной подушки.\n- Представь себе, нет. Я постоянно несколько занят.\n- Жаль... у тебя бы получилось, - продолжал детектив с непробиваемым выражением лица.\nВ такие минуты Уотсон сам не знал, чего ему хочется 

In [14]:
prompts.push_to_hub("dim/ficbook_prompts_best_10k", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [16]:
from datasets import load_dataset, Dataset, concatenate_datasets

dataset = load_dataset("dim/ficbook_prompts_best_10k")
dataset = dataset["train"]
dataset

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--ficbook_prompts_best_10k-7a1d478e1a4e866f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['prompt', 'solution_short_llama2', 'solution_full'],
    num_rows: 10000
})

In [17]:
dataset[0]

{'prompt': 'напиши фанфик с названием Метро и следующим описанием "Метро в час-пик - самый слэшерский вид транспорта."(с) Нэлт "ЛиЛ в современном мире". Или фемслэшерский?), с тегами PWP',
 'solution_short_llama2': 'Стук колес поезда отдавался в ушах гулким шумом, перекрывая собой звучащую фоном музыку. Марина зажмурилась на мгновенье, а потом снова попыталась сосредоточиться на предложении, которое читала уже в пятый раз. Буквы не желали складываться в слова. Марина вздохнула и тут же чуть не подавилась воздухом, потому что поезд вдруг начал резко тормозить. На благо, упасть в вагоне было некуда, все-таки вечерний час-пик. Девушка выдохнула, стараясь одной рукой одернуть чуть задравшуюся юбку.\n«На кой черт я ее надела?!» - Марина бросила бесплодные попытки и снова уткнулась в книжку.\nСначала она подумала, что ей показалось, что кто-то коснулся ее бедра. Ну, мало ли чего в толпе ни случается. Но уже через пару секунд девушка вздрогнула, пытаясь обернуться, но не имея такой возможност